In [2]:
#Importação das bibliotecas utilizadas
import pandas as pd
import json
import mysql.connector
import csv

#Definição do banco de dados
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database="me"
)

#Caso o teste seja executado mais de uma vez, garante que os dados serão excluídos antes de adicionar novos dados, evitando duplicidade
mycursor = mydb.cursor()
sql = "delete from logs"
mycursor.execute(sql)
mydb.commit()

#Leitura do arquivo e inserção de valores no banco de dados
#O banco possui 6 colunas (id de consumidor, id de serviço, tempo de proxy, tempo de kong, tempo de request e uma coluna log com o restando do json)
with open('logs.txt') as logs:
    for line in logs:
        line_json = json.loads(line)
        consumer_id = line_json['authenticated_entity']['consumer_id']['uuid']
        service_id = line_json['service']['id']
        l_proxy = line_json['latencies']['proxy']
        l_kong = line_json['latencies']['kong']
        l_request = line_json['latencies']['request']
        mycursor = mydb.cursor()
        sql = "INSERT INTO logs (consumer_id, service_id, log, latencies_proxy, latencies_kong, latencies_request) VALUES (%s, %s, %s, %s, %s, %s)"
        val = (consumer_id, service_id, line, l_proxy, l_kong, l_request)
        mycursor.execute(sql, val)
        mydb.commit()

In [3]:
#Criação do csv de requisições por consumidor com consulta via mysql
consumer_requests = pd.read_sql('SELECT consumer_id, count(*) as requests FROM logs group by consumer_id;', con=mydb)
consumer_requests.to_csv('consumer_requests.csv', index=False)

#Criação do csv de requisições por serviço com consulta via mysql
service_requests = pd.read_sql('SELECT service_id, count(*) as requests FROM logs group by service_id;', con=mydb)
service_requests.to_csv('service_requests.csv', index=False)

#Criação do csv de tempo médio de proxy, kong e requests por serviço com consulta via mysql
service_average = pd.read_sql('select service_id, avg(latencies_proxy) as average_proxy, avg(latencies_kong) as average_kong, avg(latencies_request) as average_request from me.logs group by service_id;', con=mydb)
service_average.to_csv('service_average.csv', index=False)